In [3]:
import optuna
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import polars as pl
import joblib

In [4]:
def build_bilstm(trial):
    model_BiLSTM = Sequential([
            Input(shape=(100, features)),


            Bidirectional(LSTM(
                units = trial.suggest_int("units_1", 64, 256, step=32),
                return_sequences=True,
                kernel_regularizer=regularizers.l1_l2(
                    l1=trial.suggest_float("l1_1_layer1", 1e-3, 3e-1),
                    l2=trial.suggest_float("l2_1_layer1", 1e-3, 3e-1),
                    ),
                recurrent_regularizer=regularizers.l1_l2(
                    l1=trial.suggest_float("l1_2_layer1", 1e-3, 3e-1),
                    l2=trial.suggest_float("l2_2_layer1", 1e-3, 3e-1),
                    ),
                )),
            Dropout(trial.suggest_float("dropout_1", 1e-5,1e-2)),
            Bidirectional(LSTM(
                units = trial.suggest_int("units_2", 64, 256, step = 32),
                return_sequences=True,
                kernel_regularizer=regularizers.l2(
                    l2 = trial.suggest_float("l2_1_layer2", 1e-3, 3e-1)
                    ),
                recurrent_regularizer=regularizers.l2(
                    l2 = trial.suggest_float("l2_2_layer2", 1e-3, 3e-1)
                    ),
                )),

            Dropout(trial.suggest_float("dropout_2", 1e-3, 3e-1)),
            Bidirectional(LSTM(
                units = trial.suggest_int("units_3", 64, 256, step = 32),
                return_sequences=True,
                kernel_regularizer=regularizers.l2(
                    l2 = trial.suggest_float("l2_1_layer3", 1e-3, 3e-1)
                    ),
                recurrent_regularizer=regularizers.l2(
                    l2 = trial.suggest_float("l2_2_layer3", 1e-3, 3e-1)
                    ),
                )) ,

            Dropout(trial.suggest_float("dropout_3", 1e-3, 3e-1)),
            Bidirectional(LSTM(
                units= trial.suggest_int("unit_4", 32,256, step = 32) ,
                return_sequences=True)),
            Dropout(trial.suggest_float("dropout_4", 1e-3, 3e-1)),
            Bidirectional(LSTM(
                units= trial.suggest_int("unit_5", 32,256, step = 32) ,
                return_sequences=True)),

            Dense(units = trial.suggest_int("Dense_1", 64,256, step = 32), activation = 'relu'),
            Dense(units = trial.suggest_int("Dense_2", 64,256, step = 32), activation = 'relu'),
            Dense(targets)
        ])
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-3, 3e-1)
    model_BiLSTM.compile(
        optimizer=Adam(learning_rate=learning_rate, clipnorm=trial.suggest_float("clip_norm", 1.0, 3.0)),
        loss=tf.keras.losses.MeanSquaredError(),
        metrics=[tf.keras.metrics.MeanSquaredError(name='mse'),
                 tf.keras.metrics.MeanAbsoluteError(name='mae'),
                ]
    )
    return model_BiLSTM

def objective(trial):
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    model = build_bilstm(trial)
    history = model.fit(
        X_train_split_reshape, y_train_split_reshape,
        validation_data=validation_data,
        epochs=100, 
        batch_size=batch_size,
        callbacks=[
            EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True),
            ReduceLROnPlateau(monitor='val_loss', factor=0.25, patience=10, min_lr=1e-5)
        ],
        verbose=0
    )
    tf.keras.backend.clear_session()
    return min(history.history["val_loss"])

In [5]:
df = pl.read_csv("/workspace/data.csv")
columns = [
'noperation','nsampling','valve','voltage','current','pressure','torque',
 'theta0','theta1','theta2',
 'cos1','cos2_inv','cos2_inv2','cos2_inv3',
 'd_valve','d_press','dv_sign','status']

X = df[columns].drop('torque')
y = df[['torque']]


features = len(X.columns)
targets = len(y.columns)


scaler = StandardScaler()
X_scaler = scaler.fit_transform(X)
X_train_split, X_val, y_train_split, y_val = train_test_split(X_scaler,y, test_size= 0.25)

X_train_split_reshape = X_train_split.reshape(X_train_split.shape[0]//100, 100, features)

X_val_reshape = X_val.reshape(X_val.shape[0]//100, 100, features)

y_train_split_reshape = y_train_split.to_numpy().reshape(y_train_split.shape[0]//100,100,targets)
y_val_reshape = y_val.to_numpy().reshape(y_val.shape[0]//100,100,targets)

validation_data = (X_val_reshape, y_val_reshape)

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20) 

[I 2025-02-21 03:40:25,544] A new study created in memory with name: no-name-ea31f337-effb-4a86-8e32-4ccfc3b6869a
2025-02-21 03:40:26.041174: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14301 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:21:00.0, compute capability: 8.6
2025-02-21 03:40:26.576948: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
/tmp/ipykernel_1165/1600206865.py:55: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-3, 3e-1)
2025-02-21 03:40:40.698409: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8905
2025-02-21 03:40:43.089700: I tensorflow/compiler/xla/se

In [ ]:
print("Best trial:", study.best_trial.params)